In [41]:
import numpy as np
import random
from collections import defaultdict


def replace_minimum_fitness_with_child(minimum, child, population, fitness_scores):
    for i in range(fitness_scores.shape[0]):
        if minimum == fitness_scores[i]:
            population[i] = child
            break


def count_queens(individual):
    count = 0
    for value in individual:
        if value != -1:
            count += 1
    return count


def get_diagonal_values(row, column):
    # there is a better way
    #   -> sum(row + col) == sum (r + c) OR abs(r - row) == abs(c - col)
    # TODO: if i have time
    
    values = []

    # get values of diagonal left up
    row_c = row
    col_c = column
    while row_c - 1 >= 0 and col_c - 1 >= 0:
        values.append([row_c - 1, col_c - 1])
        row_c -= 1
        col_c -= 1

    # get values of diagonal left down
    row_c = row
    col_c = column
    while row_c + 1 <= 7 and col_c - 1 >= 0:
        values.append([row_c + 1, col_c - 1])
        row_c += 1
        col_c -= 1

    # get values of diagonal right up
    row_c = row
    col_c = column
    while row_c - 1 >= 0 and col_c + 1 <= 7:
        values.append([row_c - 1, col_c + 1])
        row_c -= 1
        col_c += 1

    # get values of diagonal right down
    row_c = row
    col_c = column
    while row_c + 1 <= 7 and col_c + 1 <= 7:
        values.append([row_c + 1, col_c + 1])
        row_c += 1
        col_c += 1

    return np.array(values)


def check_if_queens_threaten_each_other(individual):
    # there is a better way
    # can do all of this in 2 nested fors, check column at the same time as diag
    # TODO
    
    # check for the same row -> implicitly correct since the way we represent an individual

    # check for the same column
    d = defaultdict(int)
    for col in individual:
        if col != -1:
            if d[col] == 1:
                return True
            elif d[col] == 0:
                d[col] = 1

    # check for diagonals
    for [index, number] in np.ndenumerate(individual):
        if number != -1:
            values = get_diagonal_values(index[0], number)
            for [index, number] in np.ndenumerate(individual):
                if number == -1:
                    continue
                else:
                    for val in values:
                        if val[0] == index and val[1] == number:
                            return True
    return False


def individual(number_of_queens):
    arr = np.array([-1] * LENGTH)

    while number_of_queens != 0:
        number_row = np.random.randint(0, 8)
        number_col = np.random.randint(0, 8)

        if arr[number_row] == -1:
            arr[number_row] = number_col
            number_of_queens -= 1

    return arr


def initialize_population(population_size):
    return np.array([individual(NUMBER_OF_QUEENS) for _ in range(population_size)])


def fitness_of_individual(individual):
    # score = score - number of threatening pos -> bad
    # score = score * X, where X is a scalar -> bad
    # score = abs(score - (BIGGEST_WEIGHT * scalar)) -> ok but not good
    check = check_if_queens_threaten_each_other(individual)

    score = 0
    for i in range(LENGTH):
        if individual[i] != -1:
            score += WEIGHTS[i][individual[i]]

    if check is False:
        return score
    else:
        score = abs(score - (BIGGEST_WEIGHT * 0.05))
        if score == 0:
            return 1
        else:
            return score


def fitness_of_population(population):
    scores = np.array([])
    for individual in population:
        scores = np.append(scores, fitness_of_individual(individual))

    return scores


def select_parents(population, fitness_scores):
    sum_of_fitness_scores = (fitness_scores + 1).sum()
    probability = []
    for score in fitness_scores:
        probability.append((score + 1) / sum_of_fitness_scores)
    indexes = np.random.choice(POPULATION_SIZE, size=2, replace=False, p=probability)
    return population[indexes]


def crossover(parents):
    # this is bad
    # better is to change crossover based on the ammount of queens
    #            -> if queen number is small (1, 2, 3, 4) choose ox1
    #            -> if queens number is big choose 1 point crossover
    # TODO: if i have time

    # also if population size is big -> should return only 1 child
    # if population size is small -> return 2 childs
    # TODO: if i have time
    
    crossover_point = np.random.randint(0, LENGTH - 1)
    child1 = np.concatenate((parents[1][:crossover_point], parents[0][crossover_point:]), axis=None)
    child2 = np.concatenate((parents[0][:crossover_point], parents[1][crossover_point:]), axis=None)

    while count_queens(child1) != NUMBER_OF_QUEENS and count_queens(child2) != NUMBER_OF_QUEENS:
        crossover_point = np.random.randint(0, LENGTH - 1)
        child1 = np.concatenate((parents[1][:crossover_point], parents[0][crossover_point:]), axis=None)
        child2 = np.concatenate((parents[0][:crossover_point], parents[1][crossover_point:]), axis=None)
                                
    return child1, child2


def mutate(column_of_queen, mutation_rate):
    if np.random.random() <= mutation_rate:
        column_of_queen = np.random.randint(0, LENGTH)
    return column_of_queen


def genetic_algorithm(population_size, generations, mutation_rate):
    population = initialize_population(population_size)

    for gen in range(generations):
        # Fitness evaluation
        fitness_scores = fitness_of_population(population)

        # Selection
        parents = select_parents(population, fitness_scores)

        # Crossover
        child1, child2 = crossover(parents)

        # Mutation
        mutated_child1 = np.array([mutate(chromosome, mutation_rate) if chromosome != -1 else -1 for chromosome in child1])
        mutated_child2 = np.array([mutate(chromosome, mutation_rate) if chromosome != -1 else -1 for chromosome in child2])

        fitness_score_of_mutated_child1 = fitness_of_individual(mutated_child1)
        fitness_score_of_mutated_child2 = fitness_of_individual(mutated_child2)

        # Replace the population with the new generation
        minimum = fitness_scores.min()
        if fitness_score_of_mutated_child1 > minimum:
            replace_minimum_fitness_with_child(minimum, mutated_child1, population, fitness_scores)

        if fitness_score_of_mutated_child2 > minimum:
            replace_minimum_fitness_with_child(minimum, mutated_child2, population, fitness_scores)

    return population


POPULATION_SIZE = 100

LENGTH = 8
GENERATIONS = 200
MUTATION_RATE = 0.5
NUMBER_OF_QUEENS = 8
"""
WEIGHTS = np.array([
    [100, 2, 1, 4, 5, 6, 0, 2], [0, 0, 1, 4, 1, 0, 0, 2],
    [0, 5, 1, 4, 5, 6, 0, 2], [0, 1, 0, 4, 5, 3, 0, 10],
    [0, 0, 1, 0, 5, 7, 0, 2], [0, 2, 4, 2, 4, 6, 0, 10],
    [10, 2, 5, 4, 5, 6, 0, 0], [0, 2, 14, 0, 5, 6, 100, 10]
])
"""
# 4q -> 217 max
# 5q -> 222 max

WEIGHTS = np.array([
    [100, 2, 1, 4, 5, 6, 0, 2], [0, 0, 1, 4, 1, 0, 0, 2],
    [0, 5, 1, 4, 5, 6, 0, 2], [0, 1, 100, 4, 5, 3, 0, 10],
    [0, 0, 1, 0, 5, 7, 0, 2], [0, 2, 4, 2, 4, 6, 0, 10],
    [10, 2, 5, 4, 5, 6, 0, 0], [0, 2, 14, 0, 5, 6, 4, 10]
])

# 4q -> 220 max
# 5q -> 226 max
BIGGEST_WEIGHT = np.max(WEIGHTS)

for _ in range(10):
    population = genetic_algorithm(POPULATION_SIZE, GENERATIONS, MUTATION_RATE)
    maxi = fitness_of_population(population).max()
    print(f"The best fitness score is: {maxi}")
    for p in population:
        fitness = fitness_of_individual(p)
        if fitness == maxi:
            print(f"The individual with the best fitness is: {p}")
            break

The best fitness score is: 237.0
The individual with the best fitness is: [0 3 4 2 4 7 3 2]
The best fitness score is: 227.0
The individual with the best fitness is: [0 5 3 2 5 7 5 4]
The best fitness score is: 231.0
The individual with the best fitness is: [0 3 3 2 0 2 0 2]
The best fitness score is: 227.0
The individual with the best fitness is: [0 3 4 2 2 7 0 1]
The best fitness score is: 227.0
The individual with the best fitness is: [0 4 2 2 3 7 5 2]
The best fitness score is: 225.0
The individual with the best fitness is: [0 1 0 2 1 7 0 7]
The best fitness score is: 228.0
The individual with the best fitness is: [0 3 3 2 2 0 0 2]
The best fitness score is: 230.0
The individual with the best fitness is: [0 1 3 2 5 0 0 2]
The best fitness score is: 228.0
The individual with the best fitness is: [0 1 5 2 5 7 6 7]
The best fitness score is: 232.0
The individual with the best fitness is: [0 7 4 2 7 7 3 2]
